In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import h5py
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
import time
import torch.nn.init as ini
import multiprocessing
from multiprocessing import Queue
import random
from random import shuffle
import pickle

class InputCasNet(nn.Module):
    def __init__(self):
        super(InputCasNet, self).__init__()
        self.first_upper_layer1=nn.Sequential(
            nn.Conv2d(4,64,7),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.first_upper_layer2=nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.first_under_layer1=nn.Sequential(
            nn.Conv2d(4,160,13),
            nn.BatchNorm2d(160),
            nn.ReLU()
        )
        
        self.first_final_layer=nn.Conv2d(224,5,21)
        
        self.second_upper_layer1=nn.Sequential(
            nn.Conv2d(9,64,7),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.second_upper_layer2=nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.second_under_layer1=self.under_layer1 = nn.Sequential(
            nn.Conv2d(9,160,13),
            nn.BatchNorm2d(160),
            nn.ReLU()
        )
        self.second_final_layer = nn.Conv2d(224,5,21)
    
    def forward(self, x1, x2):
        upper_x=self.first_upper_layer2(self.first_upper_layer1(x1))
        under_x=self.first_under_layer1(x1)
        x=torch.cat((upper_x, under_x), 1)
        x=self.first_final_layer(x)
        x2=torch.cat((x, x2), 1)
        upper_x2=self.second_upper_layer2(self.second_upper_layer1(x2))
        under_x2=self.second_under_layer1(x2)
        x3=torch.cat((upper_x2, under_x2), 1)
        x3=self.second_final_layer(x3)
        return x3

cas_net=InputCasNet()
x1 = Variable(torch.randn(1,4,65,65), requires_grad = True)
x2 = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = cas_net.forward(x1, x2)
print(y_pred)

Variable containing:
(0 ,0 ,.,.) = 
 -0.1743

(0 ,1 ,.,.) = 
  0.3442

(0 ,2 ,.,.) = 
 -0.1364

(0 ,3 ,.,.) = 
  0.4142

(0 ,4 ,.,.) = 
 -0.0621
[torch.FloatTensor of size 1x5x1x1]



In [2]:
f = h5py.File('/home/yiqin/train.h5','r')

#get the training set for phase 1
f_in=open("/home/yiqin/training-65x65-balanced.txt", "r")
content=f_in.readlines()
data_train_phase1=[]
data_val=[]
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    if item[0]!="HG/0001":
        data_train_phase1.append([item[0], item[1], item[2], item[3], item[4]])
    else:
        data_val.append([item[0], item[1], item[2], item[3], item[4]])
f_in.close()
print ("phase 1 data preparation process completed.")

#get the training set for phase 2 and the validation set
f_in=open("/home/yiqin/training-65x65-unbalanced.txt", "r")
content=f_in.readlines()
data_train_phase2=[]
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    if item[0]!="HG/0001":
        data_train_phase2.append([item[0], item[1], item[2], item[3], item[4]])
    else:
        data_val.append([item[0], item[1], item[2], item[3], item[4]])
f_in.close()
print ("phase 2 data preparation process completed.")

print(len(data_train_phase1))
print(len(data_train_phase2))
print(len(data_val))

phase 1 data preparation process completed.
phase 2 data preparation process completed.
8071683
21254359
1186058


In [3]:
def create_val_batch_phase1(step = 6000, key = 299):
    val_x1 = []
    val_x2 = []
    val_label = []
    batch_size = len(data_val) // step
    for i in range(batch_size):
        case,x,y,z,l = data_val.pop(i * step + key - i)
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        x1 = f[case1][case2][:, x-32:x+32+1, y-32:y+32+1, z]
        x2 = f[case1][case2][:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        val_x1.append(x1)
        val_x2.append(x2)
        val_label.append(l)
    val_x1 = torch.from_numpy(np.array(val_x1))
    val_x2 = torch.from_numpy(np.array(val_x2))
    val_label = torch.from_numpy(np.array(val_label))
    return val_x1, val_x2, val_label

In [4]:
#create valid_set
val_len = len(data_val)

val_x1, val_x2, val_y = create_val_batch_phase1()
val_y = val_y.view(-1)
val_x1=Variable(val_x1.cuda(2))
val_x2=Variable(val_x2.cuda(2))
print(len(data_val))
print(len(val_y))

1185861
197


In [13]:
#set hyper_param
learning_rate = 7e-4
l1_reg = 5e-5
optimizer = torch.optim.SGD(cas_net.parameters(), lr=learning_rate, momentum = 0.9, weight_decay = 5e-7)

In [14]:
for param in cas_net.parameters():
    if len(param.size())==4:
        ini.uniform(param, a=-5e-6, b=5e-6)

In [7]:
def create_batch(size, j):
    train_x1 = []
    train_x2 = []
    train_label = []
    for i in range(size):
        case,x,y,z,l = data_train_phase1[j * size + i]
        x,y,z,l = int(x), int(y), int(z), int(l)
        case1 = case[0:2]
        case2 = case[3:]
        x1 = f[case1][case2][:, x-32:x+32+1, y-32:y+32+1, z]
        x2 = f[case1][case2][:, x-16:x+16+1, y-16:y+16+1, z] #sample a 33x33 patch
        train_x1.append(x1)
        train_x2.append(x2)
        train_label.append(l)
    train_x1 = torch.from_numpy(np.array(train_x1))
    train_x2 = torch.from_numpy(np.array(train_x2))
    train_label = torch.from_numpy(np.array(train_label))
    return train_x1, train_x2, train_label

In [15]:
cas_net.cuda(2)
prev_time = time.clock()
num_epoch = 2
batch_size = 128
step_size = len(data_train_phase1) // batch_size
num_times=num_epoch*step_size
print(num_times)
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

126120


In [16]:
for i in range(num_epoch):
    random.shuffle(data_train_phase1)
    for j in range(step_size):
        training_x1, training_x2, training_label = create_batch(batch_size, j)
        x1_train, x2_train, y_train = Variable(training_x1.cuda(2)), Variable(training_x2.cuda(2)), Variable(training_label.cuda(2), requires_grad=False)
        y_pred = cas_net.forward(x1_train, x2_train)
        y_pred = y_pred.view(-1,5)
        loss = F.cross_entropy(y_pred, y_train)#cross entropy loss

#        l1_crit = nn.L1Loss(size_average = False)#L1 loss
#        reg_loss = 0
#        for param in net.parameters():
#            reg_loss += l1_crit(param)
#        loss+= l1_reg * reg_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #check accuracy
        index=i*step_size+j
        if index % 10 == 0:
            print ("")
            print (str(index)+' time used %.3f' % (time.clock()-prev_time))
            print ('phase 1: '+str(float(index)/num_times*100)+"% completed")
            print (loss)
            y_val_pred=cas_net.forward(val_x1, val_x2)
            y_val_pred=y_val_pred.view(-1,5)
            useless, predicted=torch.max(y_val_pred.data, 1)
            correct = (predicted == val_y.cuda(2)).sum()
            print('Validation accuracy:', float(correct)/len(val_y))
    scheduler.step()

print ("phase1: successfully trained!")
#torch.save(cas_net.state_dict(), "phase1_input_cas_net.txt")
print ("phase1: successfully saved!")


0 time used 14.510
phase 1: 0.0% completed
Variable containing:
 1.6113
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.07614213197969544

10 time used 23.574
phase 1: 0.00792895654931811% completed
Variable containing:
 1.5509
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.5786802030456852

20 time used 32.679
phase 1: 0.01585791309863622% completed
Variable containing:
 1.3850
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.7918781725888325

30 time used 41.697
phase 1: 0.02378686964795433% completed
Variable containing:
 1.4212
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.8020304568527918

40 time used 50.769
phase 1: 0.03171582619727244% completed
Variable containing:
 1.2567
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.7969543147208121

50 time used 59.791
phase 1: 0.03964478274659055% completed
Variable containing:
 1.2290
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation a


470 time used 439.104
phase 1: 0.37266095781795117% completed
Variable containing:
 0.8618
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.4517766497461929

480 time used 448.112
phase 1: 0.3805899143672693% completed
Variable containing:
 0.9197
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.6395939086294417

490 time used 457.105
phase 1: 0.3885188709165874% completed
Variable containing:
 0.9011
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.5380710659898477

500 time used 466.128
phase 1: 0.39644782746590546% completed
Variable containing:
 0.9739
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.6243654822335025

510 time used 475.132
phase 1: 0.4043767840152236% completed
Variable containing:
 0.9557
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.6294416243654822

520 time used 484.114
phase 1: 0.41230574056454167% completed
Variable containing:
 0.8290
[torch.cuda.FloatTensor of size


940 time used 862.432
phase 1: 0.7453219156359023% completed
Variable containing:
 0.7352
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.6700507614213198

950 time used 871.451
phase 1: 0.7532508721852205% completed
Variable containing:
 0.7064
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.583756345177665

960 time used 880.481
phase 1: 0.7611798287345386% completed
Variable containing:
 0.6844
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.7055837563451777

970 time used 889.500
phase 1: 0.7691087852838566% completed
Variable containing:
 0.7978
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.6142131979695431

980 time used 898.531
phase 1: 0.7770377418331748% completed
Variable containing:
 0.7284
[torch.cuda.FloatTensor of size 1 (GPU 2)]

Validation accuracy: 0.649746192893401

990 time used 907.523
phase 1: 0.784966698382493% completed
Variable containing:
 0.7972
[torch.cuda.FloatTensor of size 1 (GP

KeyboardInterrupt: 